In [ ]:
from rcwc import *
import numpy as np
import cvxpy as cp
from sklearn.neighbors import NearestNeighbors as kNN
from matplotlib import pyplot as plt
import os, time

In [ ]:
from matplotlib import cm
import matplotlib
from matplotlib.ticker import ScalarFormatter
import seaborn as sns

In [ ]:
np.random.seed(94)

In [ ]:
n = 50
pts = np.random.rand(n,2)

In [ ]:
# Building spatially correlated dataset
x = pts[:,0] + pts[:,1] - 1
colormap = matplotlib.colors.ListedColormap(sns.color_palette("coolwarm",8))
plt.scatter(pts[:,0], pts[:,1], c=x, cmap=colormap)
plt.clim(-1,1)
plt.colorbar()
plt.xlabel('x')
plt.ylabel('y', rotation='0', labelpad=20)
plt.yticks([0,0.5,1])
plt.tight_layout()

In [ ]:
# Creating edges
knearest = kNN(n_neighbors=6).fit(pts)
dists, neighbors = knearest.kneighbors(pts)
neighbors = neighbors[:,1:] #get rid of self edges

In [ ]:
# Generating samples
m = 1000 # num samples
k = 25 # size of sample
nrecruits = 2 # number of new recruits
A = np.zeros((m,n))
b = np.ones((m,n)) * 1/n
start_node = np.random.randint(low=0,high=n)
A[:,start_node] = 1

for i in range(m):
  sample = set([start_node])
  recruiters = set([start_node])
  while len(recruiters) > 0 and len(sample) < k:
    recruits = set()
    for r in recruiters:
      possible_recruits = [neigh for neigh in tuple(neighbors[r])]
      idx = np.random.choice(len(possible_recruits), size=min(k-len(sample), min(nrecruits, len(possible_recruits))))
      recruits.update([possible_recruits[id] for id in tuple(idx)])
      sample.update(recruits)
      if len(sample) >= k:
        break
    recruiters = recruits
  for elem in sample:
    A[i,elem] = 1

In [ ]:
# Plot a random sample (gold is root, red is in sample, all points are in target)
i = np.random.randint(0,m)
plt.scatter(pts[:,0], pts[:,1])
plt.scatter(pts[A[i,:] == 1,0], pts[A[i,:] == 1,1], c='red')
plt.scatter(pts[start_node,0], pts[start_node,1],c='gold')

In [ ]:
# Plot individual probability of being sampled
sample_prob = np.sum(A, axis=0) / m
mask = sample_prob > 0
plt.scatter(pts[:,0], pts[:,1], c='grey')
plt.scatter(pts[mask,0], pts[mask,1], c=sample_prob[mask])
plt.colorbar()

In [ ]:
# Basic estimator (sample mean)
a_mean = A / np.sum(A, axis=1, keepdims=True)
print('Worst-case error: {}'.format(evaluate_weights_grothendieck(a_mean, b)))
print('Spatial values error: {}'.format(evaluate_weights(a_mean, b, x)))

In [ ]:
%%time
# RCWC estimator
a_rcwc = rcwc(A, b)

In [ ]:
# Evaluating RCWC results
print('Worst-case error: {}'.format(evaluate_weights_grothendieck(a_rcwc, b, is_verbose=False)))
print('Spatial values error: {}'.format(evaluate_weights(a_rcwc, b, x)))